In [1]:
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
from tqdm.notebook import tqdm
from transformers.utils import logging
from transformers import GenerationConfig
import json
from peft import LoraConfig, get_peft_model, TaskType,PeftModel
model_name = 'google/flan-t5-xl'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 2048

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from peft import PeftModel

critque_model = PeftModel.from_pretrained(model, "trained_model/flan_t5_multi_task_1,2,3/checkpoint-4400")

In [11]:
from peft import PeftModel
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
qa_model = PeftModel.from_pretrained(model2, "trained_model/flan_t5_multi_task_4,5/checkpoint-1600")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
# %load dualt5.py
generation_config = GenerationConfig(
            temperature=0.1,
            do_sample=True,
            top_p=0.75,
            top_k=40,
            max_new_tokens=512)

import wikienv, wrappers
import re
import requests
from transformers import GenerationConfig

env = wikienv.WikiEnv()
#env = wrappers.FeverWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)



class Retriever:

    def __call__(self,term,top_sentence=12):

        return self.search_term(term,top_sentence)
        
    def step(self,env, action):
        attempts = 0
        while attempts < 10:
            try:
                return env.step(action)
            except requests.exceptions.Timeout:
                attempts += 1
    
    def search_term(self,term,top_sentence=12,count= 0):
        if count >=2:
            print("Nothing found")
            return ''
        action = f'Search[{term}]'
        action[0].lower() + action[1:]
        print(action)
        
        res = self.step(env, action[0].lower() + action[1:])[0] ## There might be some unknown unicode decoding error here
        
        if isinstance(res,str): ## Could not find but found similar term
            match = re.search(r"Similar: \[(.*?)\]", res)
            print(res)
            if match:
                list_str = match.group(1)
                # Split the list string and get the first element
                elements = re.findall(r"'(.*?)'", list_str)
                if elements:
                    first_element = elements[0]
                    print(f"Searching the most similar term {first_element} instead")
                    count+=1
                    return self.search_term(first_element,top_sentence,count)
                    
        if isinstance(res,list):
            if res[0].startswith('There were no results matching the query'): ## Nothing found
                print("Nothing found")
                return ''
            sentence = ' '.join(res[:top_sentence]).replace('\xa0','')
            context = ' '.join(sentence.split(' ')[:300]) ## Take max 300 words
            return context
        print("Unknown error",res)
        return ''
        
class DualT5:



    def __init__(self,
                 critque_model,
                 qa_model,
                 retriever,
                 t5_tokenizer,
                 generation_config=generation_config,
                 device='cuda',
                 use_critque=True,
                 max_steps=3):
        
        self.critque_model = critque_model
        self.qa_model = qa_model
        self.retriever = retriever
        self.t5_tokenizer = t5_tokenizer
        self.generation_config = generation_config
        self.max_steps=max_steps
        self.device = device
        self.traj = []
        
    def _extract_t5_for_search(self,sentence):
        ## I need to ... SEARCH[TERM]
        pattern = r'\[([^\]]+)\]' ## Check for terms inside square brackets

        match = re.search(pattern, sentence)
        if match:
        # Extract the term within square brackets
            extracted_term = match.group(1)
            return extracted_term
        else:
            return ''

    def feedback_step(self,do_feedback):
        context,query = self.traj[0]['Context'],self.traj[0]['Query']
        print("Traj length",len(self.traj))
        if len(self.traj) == 1:  ## First step, no feedback has been provided
            context_with_query = f"{context}\n{query}"
            reasoning = self.qa_step(context_with_query)
            context_with_query_reasoning = f"{context}\n{query}\n{reasoning}"
            
        else: ## The previous reasoning step and feedback is the last element of the trajectory list
            prev_step = self.traj[-1]
            
            reasoning,feedback = prev_step["Reasoning"],prev_step["Feedback"]
            context_query_feedback = f"{context}\n{query}\n{reasoning}\n{feedback}"
            print(context_query_feedback)
            reasoning = self.qa_step(context_query_feedback)
            context_with_query_reasoning = f"{context}\n{query}\nReasoning:{reasoning}"
            
        if not do_feedback:
            self.traj.append({"Reasoning":reasoning})
            return True

        feedback = self.critque_step(context_with_query_reasoning)
        
        self.traj.append({"Reasoning":reasoning,"Feedback":feedback})
        if feedback == 'FEEDBACK: No HINT':
            
            return True
            
        return False
        
    def inital_step(self,query,instruction=None):
        
        context = self.critque_step(query)
        
        if instruction:
            query = f"Instruction: {instruction}\n{query}"
            
        query_with_context = f"{context}\nQuestion:{query}"
        self.traj.append({"Context":context,"Query":query})
        
  
        
        
    def multi_step(self,query,instruction=None,do_feedback=True):
        self.traj = []
        
        self.inital_step(query,instruction=instruction)

        for i in range(2):
            end = self.feedback_step(do_feedback=do_feedback)
            if end:
                break
        ans = self.traj[-1]['Reasoning']
        return ans
            
            

                
            
            
    def qa_step(self,query):
        inputs = self.t5_tokenizer(query,return_tensors='pt')
        input_ids = inputs['input_ids'].to(self.device)
        
        with torch.no_grad():
            output = self.qa_model.generate(input_ids=input_ids,
                                                 do_sample=True,
                                                 generation_config=self.generation_config,
                                                 return_dict_in_generate=True)
            generated_output = output.sequences[0]
            s = self.t5_tokenizer.decode(generated_output,skip_special_tokens=True)
        
        return s

        
        
    def critque_step(self,query):
        ## This step returns either Context: [retrieved context] or FEEDBACK: feedback

        
        
        inputs = self.t5_tokenizer(query,return_tensors='pt')
        input_ids = inputs['input_ids'].to(self.device)
        
        with torch.no_grad():
            output = self.critque_model.generate(input_ids=input_ids,
                                                 do_sample=True,
                                                 generation_config=self.generation_config,
                                                 return_dict_in_generate=True)
            generated_output = output.sequences[0]
            s = self.t5_tokenizer.decode(generated_output,skip_special_tokens=True)
        
        search = self._extract_t5_for_search(s) ## Check if retrieval is required. This returns empty string or retrieval term
        
        if len(search)>0:  ## retrieval required
            context = self.retriever(search)

            if len(context)>0:
                self.retrieval_flag = True
               
                return f"Context: {context}"

            return f"Context: No context"
            
        return s ### This is the feedback step
        
        
    
            


In [38]:
ret = Retriever()

In [38]:
dualt5 = DualT5(critque_model,
                 qa_model,
                 ret,
                 tokenizer)

In [3]:
from datasets import load_dataset
import json
#ds = load_dataset('wiki_qa')
ds = load_dataset("json",data_files="fever_999.json")

In [ ]:
dualt5 = DualT5(critque_model,
         model2,
         ret,
         tokenizer,
         generation_config=generation_config,
         device='cuda',
         use_critque=True,
         max_steps=3)


In [5]:
generation_config = GenerationConfig(
            temperature=0.1,
            do_sample=True,
            top_p=0.9,
            top_k=30,
            n_beams=3,
            max_new_tokens=256)

In [6]:
def response_gen(text,model):
    print(text)
    input_ids = tokenizer(text,return_tensors='pt')['input_ids'].to("cuda")
    with torch.no_grad():
            output = model.generate(input_ids=input_ids,
                                                 do_sample=True,
                                                 generation_config=generation_config,
                                                 return_dict_in_generate=True)
            generated_output = output.sequences[0]
    return tokenizer.decode(generated_output)


In [44]:
def generator_gen(text,model):
    
    chat_template = [
    {"role": "system", "content": "You are an instruction following question answering model as truthfully as you can. You will make use of the context as much as possible"},
    {"role": "user", "content": f"{text}"}]
    
    input_ids = generator_tokenizer.apply_chat_template(chat_template,return_tensors='pt').to("cuda")
    with torch.no_grad():
            output = model.generate(input_ids=input_ids,
                                                 do_sample=True,
                                                 generation_config=generation_config,
                                                 return_dict_in_generate=True)
            generated_output = output.sequences[0]
    return generator_tokenizer.decode(generated_output)


In [5]:
generation_config = GenerationConfig(
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
            top_k=40,
            n_beams=2,
            max_new_tokens=512)

In [ ]:
ret('George Washington')

In [ ]:
response_gen(test,model2)

In [9]:
zero_shot_prompt_flant5 = '''Determine if a given CLAIM is SUPPORT,REFUTE or NOT ENOUGH INFO. Give only the answer.
Question: {}'''


In [16]:
def extract_ans_t5(ans):
    ans = ans.lower()
    if "not enough information" in ans:
        return 'NOT ENOUGH INFO'
    if "does not support" in ans or "refute" in ans or "no" in ans:
        return "REFUTES"
    if "support" in ans or "yes" in ans:
        return "SUPPORTS"
    return 'REFUTES'

In [13]:
from tqdm import tqdm
ans_list = []
for i in tqdm(range(333,len(ds['train']))):
    q = ds['train'][i]['question']
    ans = response_gen(zero_shot_prompt_flant5.format(q),model)
    ans_list.append(ans)
    print(ans)

  0%|                                                                                                                                                                               | 0/666 [00:00<?, ?it/s]

Determine if a given CLAIM is SUPPORT,REFUTE or NOT ENOUGH INFO. Give only the answer.
Question: L. Ron Hubbard died in September 1986.


  0%|▎                                                                                                                                                                      | 1/666 [00:02<30:57,  2.79s/it]

<pad> FEEDBACK: I will need more context to answer this question, I should search up more information on TERM[L. Ron Hubbard]</s>
Determine if a given CLAIM is SUPPORT,REFUTE or NOT ENOUGH INFO. Give only the answer.
Question: Chadwick Boseman refused to play James Brown.


  0%|▌                                                                                                                                                                      | 2/666 [00:05<29:51,  2.70s/it]

...

<pad> FEEDBACK: I will need more context to answer this question, I should search up more information on TERM[Edouard Seguin]</s>
Determine if a given CLAIM is SUPPORT,REFUTE or NOT ENOUGH INFO. Give only the answer.
Question: American Library Association is the largest of its kind and it is in the world.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 665/666 [30:45<00:02,  2.75s/it]

<pad> FEEDBACK: I will need more context to answer this question, I should search up more information on TERM[American Library Association]</s>
Determine if a given CLAIM is SUPPORT,REFUTE or NOT ENOUGH INFO. Give only the answer.
Question: US Airways Flight 1549 had 155 people on board.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666/666 [30:47<00:00,  2.77s/it]

<pad> FEEDBACK: I will need more context to answer this question, I should search up more information on TERM[Airline crash]</s>


In [18]:
import re
def extract_search(sentence):
    ## I need to ... SEARCH[TERM]
    pattern = r'\[([^\]]+)\]' ## Check for terms inside square brackets

    match = re.search(pattern, sentence)
    if match:
    # Extract the term within square brackets
        extracted_term = match.group(1)
        return extracted_term
    else:
        return ''

In [19]:
extract_search(ans)

'Airline crash'

In [80]:
ans_list[13]

'<pad> FEEDBACK: I will need more context to answer this question, I should search up more information on TERM[Aphrodite]</s>'

In [45]:
ret('See You on the Other Side')

Search[See You on the Other Side]


'See You on the Other Side is a stock phrase for farewells, especially ones where the parties may not meet each other again.. See You on the Other Side may also refer to:. Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate. HelpLearn to editCommunity portalRecent changesUpload file. Create account. Log in. Create account Log in. ItalianoPolskiSrpskohrvatski / српскохрватски. What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageGet shortened URLWikidata item. Download as PDFPrintable version. See You on the Other Side (Korn album). See You on the Other Side (Mercury Rev album).'

In [79]:
ds['train'][346]

{'id': '65902',
 'question': "Aphrodite is the daughter of Hades and Persephone in Homer's Iliad.",
 'label': 'REFUTES',
 'evidence': "[[[82466, 94249, 'Aphrodite', 7]]]",
 'pars': '["In Homer \'s Iliad , however , she is the daughter of Zeus and Dione ."]'}

In [86]:
ds['train

999

In [87]:
c = 0
for i in range(333,999):
    nested_list = eval(ds['train'][i]['evidence'])
    flattened_list = [k for sublist in nested_list for item in sublist for k in item]
    r = list(set([i.replace('_',' ') for i in flattened_list if isinstance(i,str)]))
    for item in r:
        if item == extract_search(ans_list[i-333]):
            c+=1
        else:
            

In [89]:
c/666

0.6426426426426426

['Dev Patel']

In [18]:
c = 0
for i in tqdm(range(len(ds['train']))):
    if extract_ans_t5(ans_list[i]) ==  ds['train'][i]['label']:
        c+=1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 999/999 [00:00<00:00, 27428.11it/s]


In [20]:
c/999

0.5225225225225225

['First_Red_Scare',
 'Woodrow_Wilson',
 'League_of_Nations',
 'Espionage_Act_of_1917',
 'Food_and_Fuel_Control_Act',
 'John_J._Pershing',
 'United_States_presidential_election,_1916',
 'Liberty_bond',
 'Armistice_of_11_November_1918',
 'German_Empire']